In [1]:
#########################################
##The model deals with the prediction  ##
##predict required commodities for     ##
##each county each month and the total ##
## required for the next years         ##
#########################################

In [2]:
import pandas as pd

df  = pd.read_csv('data.csv')

df.head()

,periodid,periodname,periodcode,perioddescription,dataid,dataname,datacode,datadescription,Baringo County,Bomet County,...,Siaya County,Taita Taveta County,Tana River County,Tharaka Nithi County,Trans Nzoia County,Turkana County,Uasin Gishu County,Vihiga County,Wajir County,West Pokot County
0,202201,January 2022,202201,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,2644.0,626.0,...,67556.0,496.0,4000.0,1148.0,4598.0,31277.0,3127.0,29950.0,1146.0,8189.0
1,202202,February 2022,202202,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,3529.0,444.0,...,52890.0,983.0,2889.0,1398.0,5390.0,23382.0,2006.0,20458.0,1197.0,6908.0
2,202203,March 2022,202203,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,2998.0,460.0,...,50940.0,1015.0,2392.0,671.0,6367.0,19729.0,2093.0,19161.0,885.0,6343.0
3,202204,April 2022,202204,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,2755.0,508.0,...,41512.0,698.0,1567.0,509.0,5187.0,22980.0,1221.0,18753.0,936.0,9657.0
4,202205,May 2022,202205,NaN,cPlWFYbBacW.w77uMi1KzOH,MOH 743 Rev2020_Rapid Diagnostic Tests Total Q...,NaN,NaN,2878.0,355.0,...,63304.0,748.0,2239.0,1507.0,11820.0,35963.0,1595.0,34762.0,1236.0,13388.0


In [16]:
#create a timestamp(the format is YYYY-MM-DD)
datetime  = []

def get_datetime():

    for timestamp in df['periodid']:

        datetime.append(str(timestamp)[:4]+"-"+str(timestamp)[-2:]+"-01")

get_datetime()
#print(len(datetime))


In [17]:
#per county and loop for all
#rename the columns
new_names  = {"Baringo County": 'baringo', "Bomet County": 'bomet', "Bungoma County": 'bungoma', "Busia County": 'busia', "Elgeyo Marakwet County": 'elgeyo',
               "Embu County": 'embu', "Garissa County": 'garissa', "Homa Bay County": 'homabay', "Isiolo County": 'isiolo', "Kajiado County": 'kajiado',
                "Kakamega County": 'kakamega', "Kericho County": 'kericho', "Kiambu County": 'kiambu', "Kilifi County": 'kilifi', "Kirinyaga County": 'kirinyaga',
                "Kisii County": 'kisii', "Kisumu County": 'kisumu', "Kitui County": 'kitui', "Kwale County": 'kwale', "Laikipia County": 'laikipia', "Lamu County": 'lamu',
                "Machakos County": 'machakos', "Makueni County": 'makueni', "Mandera County": 'mandera', "Marsabit County": 'marsabit', "Meru County": 'meru', "Migori County": 'migori',
                "Mombasa County": 'mombasa', "Muranga County": 'muranga', "Nairobi County": 'nairobi', "Nakuru County": 'nakuru', "Nandi County": 'nandi', "Narok County": 'narok',
                "Nyamira County": 'nyamira', "Nyandarua County": 'nyandarua', "Nyeri County": 'nyeri', "Samburu County": 'samburu', "Siaya County": 'siaya', "Taita Taveta County": 'taita', 
                "Tana River County": 'tanariver', "Tharaka Nithi County": 'tharaka', "Trans Nzoia County": 'transzoia', "Turkana County": 'turkana', "Uasin Gishu County": 'uasingishu',
                "Vihiga County": 'vihiga', "Wajir County": 'wajir', "West Pokot County": 'westpokot'}

df.rename(columns=new_names, inplace=True)

years = [ periodid for periodid in df['periodid']]

In [18]:
#create a data to be supported during prediction(the format is y)
county_monthly_data = []

def county_year_consumption(county):

    """This county_year_consumption function gives out the total commodities dispatched a certain complete year
       for a particular county
    """
    #run above function to get the year periods in place

    county_monthly_data.clear()
    try:
      for periodid in years:
         val = df.loc[df['periodid']==int(periodid)].sum()
         
         commodity_total = val[county]
         county_monthly_data.append(commodity_total)
         #print(f"{county} has {commodity_total} kits in {periodid}")
      
       
    except Exception as e:
       
       if e.__class__.__name__ == 'KeyError':
          print("Check if the county name is accurate.Or ask the lead developer for solution.")

county_year_consumption('nakuru')

#print(len(county_monthly_data))

In [19]:
#create a dataframe
#Have the data in a dictionary 
#the first key will be ds while the other y

dataframe  = {'ds':datetime , 'y': county_monthly_data}

new_df = pd.DataFrame(dataframe)

new_df.head()

,ds,y
0,2022-01-01,998.0
1,2022-02-01,1159.0
2,2022-03-01,1268.0
3,2022-04-01,909.0
4,2022-05-01,1079.0
